# Knowledge Distillation with PyTorch (MNIST Classification)

This repository implements **Knowledge Distillation** in PyTorch using the **MNIST dataset**. The goal is to train a **Student model** to achieve similar accuracy to a **Teacher model** while being significantly smaller and more efficient.

## Overview

* A **Teacher model** (larger network) is trained on MNIST.
* A **Student model** (smaller network) learns from both the **Teacher's soft labels** and the ground-truth labels using **Distillation Loss**.
* The **Student model** achieves **~97.68% accuracy**, nearly matching the **Teacher's 97.69% accuracy**, with **73.5% fewer parameters**.

## Model Architectures

| Model | Architecture (Fully Connected Layers) | Parameters | Accuracy (%) |
|-------|--------------------------------------|------------|--------------|
| **Teacher** | FC(784 → 1200) → FC(1200 → 1200) → FC(1200 → 10) | **2,395,210** | **97.69%** |
| **Student** | FC(784 → 800) → FC(800 → 10) | **636,010** (**73.5% smaller**) | **97.68%** |




### Train the Student Model with Distillation
* Learns from **soft labels** (teacher's predictions) and **ground-truth labels**.
* Uses **Distillation Loss**: $\text{Loss} = \alpha \cdot \text{KL-Divergence} + (1 - \alpha) \cdot \text{CrossEntropyLoss}$
* **Temperature (T=5.0)** controls softening of teacher logits.
* **Alpha (α=0.7)** balances hard vs. soft targets.

## Results

### Performance Comparison

| Model | Accuracy (%) | Parameters | Size Reduction (%) |
|-------|--------------|------------|---------------------|
| **Teacher** | **97.69%** | **2,395,210** | - |
| **Student** | **97.68%** | **636,010** | **73.5% smaller** |

* **Minimal Accuracy Drop (~0.01%)**
* **4x Reduction in Model Size**


## References

* Hinton et al., "Distilling the Knowledge in a Neural Network" (2015)


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:
class TeacherModel(nn.Module):
    def __init__(self):
        super(TeacherModel, self).__init__()
        self.fc1 = nn.Linear(28*28, 1200)
        self.fc2 = nn.Linear(1200, 1200)
        self.fc3 = nn.Linear(1200, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.fc1 = nn.Linear(28*28, 800)
        self.fc2 = nn.Linear(800, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 13.2MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 352kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 3.27MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 6.92MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher = F.softmax(teacher_logits / temperature, dim=1)
    soft_student = F.log_softmax(student_logits / temperature, dim=1)
    kl_div = F.kl_div(soft_student, soft_teacher, reduction='batchmean') * (temperature ** 2)
    ce_loss = F.cross_entropy(student_logits, labels)
    return alpha * kl_div + (1 - alpha) * ce_loss

In [7]:
teacher = TeacherModel()
optimizer = optim.Adam(teacher.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [8]:
def train_teacher(model, train_loader, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        print(f"Teacher Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

train_teacher(teacher, train_loader, optimizer, criterion)

student = StudentModel()
optimizer = optim.Adam(student.parameters(), lr=0.001)

def train_student(student, teacher, train_loader, optimizer, temperature=5.0, alpha=0.7, epochs=5):
    student.train()
    teacher.eval()
    for epoch in range(epochs):
        for data, target in train_loader:
            optimizer.zero_grad()
            student_logits = student(data)
            with torch.no_grad():
                teacher_logits = teacher(data)
            loss = distillation_loss(student_logits, teacher_logits, target, temperature, alpha)
            loss.backward()
            optimizer.step()
        print(f"Student Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

train_student(student, teacher, train_loader, optimizer)

Teacher Epoch 1/5, Loss: 0.12028646469116211
Teacher Epoch 2/5, Loss: 0.22822067141532898
Teacher Epoch 3/5, Loss: 0.017489513382315636
Teacher Epoch 4/5, Loss: 0.19024941325187683
Teacher Epoch 5/5, Loss: 0.028011934831738472
Student Epoch 1/5, Loss: 0.6762599945068359
Student Epoch 2/5, Loss: 0.19092245399951935
Student Epoch 3/5, Loss: 0.2055620551109314
Student Epoch 4/5, Loss: 0.23702338337898254
Student Epoch 5/5, Loss: 0.1306477189064026


In [9]:
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy

print("Teacher Model Accuracy:")
teacher_accuracy = evaluate(teacher, test_loader)

print("Student Model Accuracy:")
student_accuracy = evaluate(student, test_loader)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

Teacher Model Accuracy:
Accuracy: 97.64%
Student Model Accuracy:
Accuracy: 97.51%


In [10]:
teacher_params = count_parameters(teacher)
student_params = count_parameters(student)

print(f"Teacher Model Parameters: {teacher_params}")
print(f"Student Model Parameters: {student_params}")

print(f"Teacher Accuracy: {teacher_accuracy:.2f}%, Parameters: {teacher_params}")
print(f"Student Accuracy: {student_accuracy:.2f}%, Parameters: {student_params}")

Teacher Model Parameters: 2395210
Student Model Parameters: 636010
Teacher Accuracy: 97.64%, Parameters: 2395210
Student Accuracy: 97.51%, Parameters: 636010
